# Notebook 4 - NLP Grundlagen


## 1) Lernziele
- Du vergleichst Bag of Words und TF-IDF auf derselben Textaufgabe.
- Du verstehst Tokenisierung, Vokabularaufbau und Vektorisierung als Minimalmodell.
- Du berechnest TF-IDF an einem kleinen Beispiel nachvollziehbar per Handformel.
- Du testest Embeddings spielerisch und unterscheidest Wort- von Satzreprasentationen.
- Du setzt einfache Erweiterungen fuer Klassifikation und Sentiment um.


## 2) Warm-up Spielzelle: Bag of Words und TF-IDF mit Aehnlichkeit
- Mini-Korpus plus neuer Satz als Eingabe.
- Methoden: Count oder TF-IDF.
- Ausgabe: Top Features, Aehnlichkeitstabelle, TF-IDF-Heatmap.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.linear_model import LogisticRegression

CORPUS = [
    "Der Roboter sortiert Pakete im Lager.",
    "Das Expertensystem nutzt klare Regeln.",
    "Maschinelles Lernen findet Muster in Daten.",
    "Heute ist das Wetter warm und sonnig.",
    "Wir ueben Textklassifikation mit Python.",
    "TF-IDF gewichtet informative Begriffe staerker.",
    "Bag of Words ignoriert oft Reihenfolge.",
    "Der Hund rennt schnell durch den Garten.",
    "Semantik ist mit reinen Wortzaehlungen schwer.",
    "Kurze Saetze helfen beim Einstieg in NLP.",
]

STOPWORDS_DE = {
    "der", "die", "das", "und", "ist", "im", "in", "mit", "wir", "den", "ein", "eine",
    "heute", "oft", "durch", "zu", "von", "am", "an", "auf", "beim"
}

print('Korpusgroesse:', len(CORPUS), 'Saetze')


In [ ]:
def make_vectorizer(method, ngram_max, use_stopwords, min_df):
    stop_words = list(STOPWORDS_DE) if use_stopwords else None
    kwargs = {
        'ngram_range': (1, ngram_max),
        'min_df': min_df,
        'stop_words': stop_words,
    }
    if method == 'Count':
        return CountVectorizer(**kwargs)
    return TfidfVectorizer(**kwargs)


def top_features(matrix, feature_names, k=10):
    scores = np.asarray(matrix.sum(axis=0)).ravel()
    idx = np.argsort(scores)[::-1]
    top_idx = idx[:min(k, len(idx))]
    return [(feature_names[i], float(scores[i])) for i in top_idx if scores[i] > 0]


def plot_tfidf_heatmap(feature_names, top_k=20):
    # Fuer die Heatmap nehmen wir immer TF-IDF, auch wenn Count gewaehlt wurde.
    tfidf = TfidfVectorizer(vocabulary=feature_names)
    X_tfidf = tfidf.fit_transform(CORPUS)

    importance = np.asarray(X_tfidf.sum(axis=0)).ravel()
    idx = np.argsort(importance)[::-1][:min(top_k, len(importance))]
    selected_features = [tfidf.get_feature_names_out()[i] for i in idx]
    mat = X_tfidf[:, idx].toarray()

    plt.figure(figsize=(10, 4.2))
    im = plt.imshow(mat, aspect='auto', cmap='YlGnBu')
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.yticks(range(len(CORPUS)), [f'D{i+1}' for i in range(len(CORPUS))])
    plt.xticks(range(len(selected_features)), selected_features, rotation=45, ha='right')
    plt.title('TF-IDF Matrix Heatmap (Korpus x Top Features)')
    plt.tight_layout()
    plt.show()


method_dd = widgets.Dropdown(options=['Count', 'TFIDF'], value='Count', description='method')
ngram_sl = widgets.IntSlider(value=1, min=1, max=2, step=1, description='ngram_max', continuous_update=False)
stop_cb = widgets.Checkbox(value=False, description='use_stopwords')
min_df_sl = widgets.IntSlider(value=1, min=1, max=3, step=1, description='min_df', continuous_update=False)
new_text = widgets.Text(value='Das System erkennt Muster in Textdaten.', description='neuer Satz', layout=widgets.Layout(width='95%'))
run_btn = widgets.Button(description='Run warm-up', button_style='info')
out = widgets.Output()


def run_warmup(_):
    with out:
        out.clear_output()
        vec = make_vectorizer(method_dd.value, ngram_sl.value, stop_cb.value, min_df_sl.value)

        all_docs = CORPUS + [new_text.value.strip()]
        X_all = vec.fit_transform(all_docs)
        feature_names = vec.get_feature_names_out()

        X_corpus = X_all[:-1]
        x_new = X_all[-1]

        tf = top_features(X_corpus, feature_names, k=10)
        print('Top Features:')
        if tf:
            for token, val in tf:
                print(f'- {token}: {val:.4f}')
        else:
            print('- Keine Features gefunden (min_df/stopwords pruefen).')

        sims = cosine_similarity(x_new, X_corpus).ravel()
        sim_df = pd.DataFrame({
            'satz_id': [f'S{i+1}' for i in range(len(CORPUS))],
            'satz': CORPUS,
            'cosine_similarity': sims,
        }).sort_values('cosine_similarity', ascending=False).reset_index(drop=True)

        print('\nAehnlichkeit neuer Satz vs Korpus:')
        display(sim_df)

        plot_tfidf_heatmap(feature_names, top_k=20)


run_btn.on_click(run_warmup)

display(widgets.VBox([
    method_dd,
    ngram_sl,
    stop_cb,
    min_df_sl,
    new_text,
    run_btn,
    out,
]))


## 3) Minimalmodell und Pseudocode (Token, Vokabular, Vektor)
- Tokenisierung zerlegt Saetze in Grundbausteine.
- Das Vokabular sammelt alle beobachteten Begriffe.
- Jeder Satz wird als Vektor ueber dem Vokabular dargestellt.

```text
INPUT: dokumente
tokens_pro_doc <- tokenize(dokumente)
vocab <- sort(unique(alle_tokens))
FOR doc IN tokens_pro_doc
  vec <- nullvektor(len(vocab))
  FOR token IN doc
    vec[index(vocab, token)] += 1   # oder TF-IDF-Gewicht
RETURN vocab, matrix_aus_vektoren
```


In [ ]:
# Minimalbeispiel mit explizitem Vokabular und Zaehlern.
mini_docs = [
    'katze mag milch',
    'hund mag park',
    'katze und hund'
]

# Sehr einfache Tokenisierung via split.
tokens = [d.split() for d in mini_docs]
vocab = sorted({t for doc in tokens for t in doc})

rows = []
for d, tok in zip(mini_docs, tokens):
    vec = {v: 0 for v in vocab}
    for t in tok:
        vec[t] += 1
    vec['doc'] = d
    rows.append(vec)

display(pd.DataFrame(rows)[['doc'] + vocab])


## 4) TF-IDF Rechenbeispiel plus Visualisierung


In [ ]:
# Kleiner Datensatz mit 3 Dokumenten und 3 Zielbegriffen.
docs = [
    'ki modell daten daten',
    'daten analyse statistik',
    'modell lernen ki',
]
terms = ['ki', 'daten', 'modell']
N = len(docs)

rows = []
for i, d in enumerate(docs, 1):
    toks = d.split()
    for term in terms:
        tf = toks.count(term)
        df = sum(1 for doc in docs if term in doc.split())
        idf = np.log((N + 1) / (df + 1)) + 1
        tfidf = tf * idf
        rows.append({'doc': f'D{i}', 'term': term, 'tf': tf, 'df': df, 'idf': round(idf, 4), 'tfidf': round(tfidf, 4)})

calc_df = pd.DataFrame(rows)
display(calc_df)

# Balkenplot: mittleres TF-IDF je Begriff.
mean_tfidf = calc_df.groupby('term', as_index=False)['tfidf'].mean()
plt.figure(figsize=(5.4, 3.5))
plt.bar(mean_tfidf['term'], mean_tfidf['tfidf'], color='tab:blue')
plt.title('Mittleres TF-IDF fuer 3 Begriffe')
plt.ylabel('mean tfidf')
plt.tight_layout()
plt.show()


## 5) Embeddings Block: Word und Satz-Embeddings (spielbar)
### BoW vs Embeddings
- BoW/TF-IDF modelliert Woerter als diskrete Merkmale ohne tiefe Semantik.
- Embeddings legen Woerter/Saetze in kontinuierliche Vektorraeume.
- Semantisch aehnliche Begriffe liegen im Embeddingraum naeher beieinander.
- Satz-Embeddings komprimieren ganze Saetze fuer Suche und Aehnlichkeitsvergleiche.
- Kontext bleibt bei einfachen statischen Wortvektoren begrenzt.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Versuche leichte vortrainte Embeddings aus gensim zu laden.
# Falls nicht verfuegbar: SVD auf TF-IDF als Pseudo-Embedding (Approximation).
REAL_EMBEDDINGS = None
EMBEDDING_BACKEND = 'pseudo_svd'

try:
    import gensim.downloader as api
    REAL_EMBEDDINGS = api.load('glove-wiki-gigaword-50')
    EMBEDDING_BACKEND = 'gensim_glove_50'
except Exception:
    REAL_EMBEDDINGS = None
    EMBEDDING_BACKEND = 'pseudo_svd'

print('Embedding Backend:', EMBEDDING_BACKEND)

# Basis fuer Pseudo-Embeddings
pseudo_texts = CORPUS + [
    'robot', 'system', 'learning', 'data', 'dog', 'weather', 'text', 'python', 'rules', 'model'
]
pseudo_tfidf = TfidfVectorizer(min_df=1)
X_pseudo = pseudo_tfidf.fit_transform(pseudo_texts)
svd = TruncatedSVD(n_components=min(20, X_pseudo.shape[1]-1), random_state=0)
X_emb = svd.fit_transform(X_pseudo)

pseudo_index = {txt: i for i, txt in enumerate(pseudo_texts)}


def get_word_vector(word):
    w = word.strip().lower()
    if REAL_EMBEDDINGS is not None and w in REAL_EMBEDDINGS:
        return REAL_EMBEDDINGS[w]
    if w in pseudo_index:
        return X_emb[pseudo_index[w]]
    return None


def sentence_vector(sentence):
    if REAL_EMBEDDINGS is not None:
        toks = [t.lower().strip('.,!?') for t in sentence.split()]
        vecs = [REAL_EMBEDDINGS[t] for t in toks if t in REAL_EMBEDDINGS]
        if vecs:
            return np.mean(vecs, axis=0)
    # Fallback: SVD-Vektor aus TF-IDF-Satzdarstellung
    mat = pseudo_tfidf.transform([sentence])
    return svd.transform(mat)[0]


def safe_cos(a, b):
    if a is None or b is None:
        return np.nan
    return float(cosine_similarity([a], [b])[0, 0])


def top5_similar_words(word):
    if REAL_EMBEDDINGS is None:
        return []
    w = word.strip().lower()
    if w not in REAL_EMBEDDINGS:
        return []
    return REAL_EMBEDDINGS.most_similar(w, topn=5)


def plot_word_projection(words):
    vecs = []
    valid_words = []
    for w in words:
        v = get_word_vector(w)
        if v is not None:
            vecs.append(v)
            valid_words.append(w)
    if len(valid_words) < 2:
        print('Zu wenige bekannte Woerter fuer Projektion.')
        return

    mat = np.vstack(vecs)
    pca = PCA(n_components=2, random_state=0)
    pts = pca.fit_transform(mat)

    plt.figure(figsize=(6.2, 4.2))
    plt.scatter(pts[:, 0], pts[:, 1], color='tab:blue')
    for i, w in enumerate(valid_words):
        plt.text(pts[i, 0], pts[i, 1], w)
    plt.title('2D Projektion von Wortvektoren (PCA)')
    plt.tight_layout()
    plt.show()


mode_dd = widgets.Dropdown(options=['word', 'sentence'], value='word', description='embedding_mode')
word1_in = widgets.Text(value='robot', description='word1')
word2_in = widgets.Text(value='system', description='word2')
sent1_in = widgets.Text(value='Das System lernt aus Daten.', description='sentence1', layout=widgets.Layout(width='95%'))
sent2_in = widgets.Text(value='Modelle erkennen Muster.', description='sentence2', layout=widgets.Layout(width='95%'))
compare_btn = widgets.Button(description='Compare', button_style='info')
emb_out = widgets.Output()


def on_compare(_):
    with emb_out:
        emb_out.clear_output()

        if mode_dd.value == 'word':
            w1, w2 = word1_in.value.strip(), word2_in.value.strip()
            v1, v2 = get_word_vector(w1), get_word_vector(w2)
            sim = safe_cos(v1, v2)
            print(f'Word cosine similarity ({w1}, {w2}):', sim)

            if REAL_EMBEDDINGS is not None:
                sim_words = top5_similar_words(w1)
                sim_df = pd.DataFrame(sim_words, columns=['word', 'score'])
                print('\nTop 5 aehnliche Woerter zu', w1)
                display(sim_df)
            else:
                print('\nTop-5 Wortnachbarn nur mit echten Embeddings verfuegbar (Fallback aktiv).')

            plot_word_projection(['robot', 'system', 'learning', 'data', 'dog', 'weather', 'text', 'python', 'rules', 'model'])

        else:
            s1, s2 = sent1_in.value.strip(), sent2_in.value.strip()
            v1, v2 = sentence_vector(s1), sentence_vector(s2)
            sim = safe_cos(v1, v2)
            print('Sentence cosine similarity:', sim)
            print('Hinweis: Satz-Embeddings sind kompakte Repraesentationen ganzer Saetze, gut fuer Aehnlichkeit und Suche.')


compare_btn.on_click(on_compare)

display(widgets.VBox([
    mode_dd,
    word1_in,
    word2_in,
    sent1_in,
    sent2_in,
    compare_btn,
    emb_out,
]))


## 6) Aufsteigende Erweiterungen: Klassifikation, Sentiment, Themen


In [ ]:
# Erweiterung 1: Mini-Klassifikation (2 Labels) auf TF-IDF + Logistic Regression.
cls_texts = [
    'Das Modell erkennt Muster gut.',
    'Wir trainieren einen Klassifikator.',
    'Der Hund spielt im Park.',
    'Heute scheint die Sonne.',
    'NLP nutzt Vektoren fuer Text.',
    'Regeln koennen Entscheidungen erklaeren.',
]
cls_y = [1, 1, 0, 0, 1, 1]

vec = TfidfVectorizer(min_df=1)
X = vec.fit_transform(cls_texts)
clf = LogisticRegression(max_iter=1000)
clf.fit(X, cls_y)
pred = clf.predict(X)
acc = (pred == np.array(cls_y)).mean()

print('Mini-Klassifikation Accuracy (Train-Set-Demo):', round(float(acc), 4))


In [ ]:
# Erweiterung 2: Mini-Sentiment mit handgemachtem Set.
sent_texts = [
    'Das Produkt ist grossartig und hilfreich.',
    'Ich mag die klare Struktur.',
    'Die Ergebnisse sind schlecht und verwirrend.',
    'Das war eine tolle Erklaerung.',
    'Ich bin unzufrieden mit der Qualitaet.',
    'Der Ablauf war angenehm und schnell.',
    'Das Update ist problematisch und langsam.',
    'Sehr gute Dokumentation und Beispiele.',
    'Ich finde es frustrierend.',
    'Das Tool ist praktisch und stabil.',
]
sent_y = [1, 1, 0, 1, 0, 1, 0, 1, 0, 1]

sv = TfidfVectorizer(min_df=1)
Xs = sv.fit_transform(sent_texts)
mdl = LogisticRegression(max_iter=1000)
mdl.fit(Xs, sent_y)
pred_s = mdl.predict(Xs)
acc_s = (pred_s == np.array(sent_y)).mean()

res_df = pd.DataFrame({'text': sent_texts, 'true': sent_y, 'pred': pred_s})
print('Mini-Sentiment Accuracy (Train-Set-Demo):', round(float(acc_s), 4))
display(res_df.head(6))


### Erweiterung 3: Themen (Konzept)
- Themenmodellierung gruppiert Dokumente ueber wiederkehrende Wortmuster.
- Typische Verfahren sind LSA/NMF/LDA mit unterschiedlichen Annahmen.
- Fuer dieses Notebook bleibt es beim Konzept, um Rechenaufwand klein zu halten.
- In der Praxis folgt nach Themen oft eine manuelle Label-Interpretation.


## 7) Mini Leitfaden (7 bis 10 Minuten)
- Minute 0-1: Lernziele lesen und Korpus ueberfliegen.
- Minute 1-3: Warm-up mit Count starten, Aehnlichkeitstabelle lesen.
- Minute 3-5: auf TF-IDF wechseln und Heatmap vergleichen.
- Minute 5-7: TF-IDF Mini-Rechnung durchgehen und Balkenplot interpretieren.
- Minute 7-9: Embeddings-Block in word und sentence mode testen.
- Minute 9-10: Erweiterungen kurz laufen lassen und Grenzen notieren.


## Mini Uebungen
1. Formuliere zwei neue Saetze, die inhaltlich aehnlich sind, aber unterschiedliche Woerter nutzen, und vergleiche BoW vs Embeddings.
2. Erhoehe `min_df` und dokumentiere, welche Features aus der Heatmap verschwinden.
3. Setze `ngram_max=2` und identifiziere ein hilfreiches Bigramm.
4. Aendere im TF-IDF Rechenbeispiel ein Dokument und berechne den Effekt auf `idf` fuer `daten`.
5. Ergaenze zwei Sentiment-Saetze und pruefe, wie stabil die Mini-Klassifikation bleibt.
